In [105]:
#Função que retorna TRUE ou FALSE para contabilizar (ou não) potências negativas

def prop(v1, v2):
    p = 100
    if (v1 == v2):
        p = 0
    if (v1 > v2) and (v1 != 0):
        p = abs(100-((100*v2)/v1))
    if (v2 > v1) and (v2 != 0):
        p = abs(100-((100*v1)/v2))
    return p

def potNegativa(customerType, nElem, vA, vB, vC, avA, avB, avC, iA, iB, iC, aiA, aiB, aiC, fpA, fpB, fpC, pA, pB, pC):
    #customerType pode existir em apenas dois valores
    #  "BT" - para qualquer medição em baixa-tensão
    #  "MT" - para qualquer medição em média-tensão
    #nElem é o numero de elementos de medição (em geral, MT pode ser 3 ou 2, BT (BTI/BT200A) normalmente é 3)
    
    result = False
    if (customerType == "MT"):
        #considera as potências no secundário
        maxPotNegMT = -8 #Watts
        minFPPotNeg = abs(-0.2)
        if ((pA <= maxPotNegMT and abs(fpA) > minFPPotNeg) or (pB <= maxPotNegMT and abs(fpB) > minFPPotNeg)) or ((nElem == 3 and (pC <= maxPotNegMT and abs(fpC) > minFPPotNeg)) or (nElem == 2 and (pC <= maxPotNegMT and abs(fpC) > minFPPotNeg) and (aiC < 30 or aiC > 210))):
            result = True
    if (customerType == "BT"):
        maxPotNegMT = -20 #Watts
        minFPPotNeg = -0.4
        maxPropCorr = 50
        maxAmpere = 0.99
        if (pA <= maxPotNegMT or pB <= maxPotNegMT or pC <= maxPotNegMT):
            #qualquer das tres potencias negativas em principio é para contabilizar
            #só será desarmado se tratar-se de Kirchoff para cargas equilibradas com banco de capacitor e carga não trifasica pequena
            result = True

            if (pA != 0):
                spA = pA/abs(pA)
            else:
                spA = +1
            if (pB != 0):
                spB = pB/abs(pB)
            else:
                spB = +1
            if (pC != 0):
                spC = pC/abs(pC)
            else:
                spC = +1
            if (fpA != 0):
                sfpA = (fpA/abs(fpA)) * spA
            else:
                sfpA = +1
            if (fpB != 0):
                sfpB = (fpB/abs(fpB)) * spB
            else:
                sfpB = +1
            if (fpC != 0):
                sfpC = (fpC/abs(fpC)) * spC
            else:
                sfpC = +1
            if (sfpA == -1 and sfpB == -1 and sfpC == -1):
                #três fatores de potencia sao capacitivos e pequenos
                if ((abs(fpA)*sfpA) > minFPPotNeg and (abs(fpB)*sfpB) > minFPPotNeg and (abs(fpC)*sfpC) > minFPPotNeg):
                    #quantas fases negativas (só pode haver 1)
                    fasesNeg = 0
                    if (spA < 0): 
                        fasesNeg += 1
                    if (spB < 0): 
                        fasesNeg += 1
                    if (spC < 0): 
                        fasesNeg += 1
                    if (fasesNeg == 1):
                        #tres correntes precisam estar relativamente equilibradas
                        propiAB = prop(iA, iB)
                        propiAC = prop(iA, iC)
                        propiBC = prop(iB, iC)
                        if (iA <= maxAmpere and iB <= maxAmpere and iC <= maxAmpere and propiAB < maxPropCorr and propiAC < maxPropCorr and propiBC < maxPropCorr):
                            result = False

    #retorna o resultado
    return result


In [106]:
#MT
res =  potNegativa("MT", 3, 117, 118, 117, 0, 120, 240, 0.01, 0.02, 0.01, 92, 210, 330, 0.2, -0.001, -0.001, -12, 1, 0)
print (res)

False


In [107]:
#BT
res =  potNegativa("BT", 3, 117, 118, 117, 0, 120, 240, 0.76, 0.44, 0.46, 92, 210, 330, 0.1, -0.1, -0.1, -12, 1, 1)
print (res)

False


In [108]:
#teste BT BTI411387957
import numpy as np
import pandas as pd

file = 'fasor_08432289_20190625_110427.csv'
grid=pd.read_csv(file,delimiter=';').astype('object')

In [109]:
grid.head()

,Data,Dia,vA,vB,vC,avA,avB,avC,iA,iB,...,AC,Distorcao Harmonica (%),Frequencia (Hz),Sequencia,1,2,3,1.1,2.1,3.1
0,01/12/2018 00:00,Sabado,135,132,131,0,120,240,0.48,0.46,...,230,0,60,132,44,0,251,12.383.964.0000,547,27.101.060.0000
1,01/12/2018 00:15,Sabado,133,132,132,0,120,240,0.47,0.46,...,229,0,60,132,44,0,250,12.384.008.0000,547,27.101.312.0000
2,01/12/2018 00:30,Sabado,134,131,132,0,120,240,0.47,0.46,...,230,0,60,132,43,0,250,12.384.051.0000,547,27.101.560.0000
3,01/12/2018 00:45,Sabado,132,133,132,0,120,240,0.46,0.44,...,228,0,60,132,46,0,246,12.384.097.0000,547,27.101.808.0000
4,01/12/2018 01:00,Sabado,133,132,133,0,120,240,0.47,0.42,...,230,0,60,132,43,0,249,12.384.140.0000,547,27.102.056.0000


In [110]:
for index, row in grid.iterrows():
    data = row['Data']
    vA = float(row['vA'])
    vB = float(row['vB'])
    vC = float(row['vC'])
    avA = float(row['avA'])
    avB = float(row['avB'])
    avC = float(row['avC'])
    iA = float(row['iA'])
    iB = float(row['iB'])
    iC = float(row['iC'])
    aiA = float(row['aiA'])
    aiB = float(row['aiB'])
    aiC = float(row['aiC'])
    pA = float(row['pA'])
    pB = float(row['pB'])
    pC = float(row['pC'])
    fpA = float(row['fpA'])
    fpB = float(row['fpB'])
    fpC =  float(row['fpC'])

    res = potNegativa("BT", 3, vA, vB, vC, avA, avB, avC, iA, iB, iC, aiA, aiB, aiC, fpA, fpB, fpC, pA, pB, pC)
    if (res):
        print (data)
